In [1]:
import pandas as pd
import numpy as np
import censusgeocode as cg
import time
from datetime import datetime

import geopandas as gpd
import os

In [2]:
keep_col = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)',
            'Provider First Line Business Practice Location Address',
            'Provider Business Practice Location Address City Name',
            'Provider Business Practice Location Address State Name',
            'Provider Business Practice Location Address Postal Code',
            'Provider Business Practice Location Address Telephone Number']

taxon_codes = ['Healthcare Provider Taxonomy Code_' + str(i+1) for i in range(15)]
keep_col += taxon_codes
community_pharm = ['222Z00000X', '224P00000X','335E00000X']
npi_csv = 'npidata_pfile_20050523-20230212.csv'

def csv_chunks(file,chunk_size,keep_cols,row_sub):
    header_fields = list(pd.read_csv(npi_csv, nrows=1))
    header_locs = [header_fields.index(i) for i in keep_cols]
    skip = 1
    it_n = 0
    sub_n = 0
    ret_chunk = chunk_size
    fin_li_dat = []
    while ret_chunk == chunk_size:
        file_chunk = pd.read_csv(file, usecols=header_locs, skiprows=skip, 
                     nrows=chunk_size, names=header_fields, dtype='str')
        sub_dat = row_sub(file_chunk)
        fin_li_dat.append( sub_dat.copy() )
        skip += chunk_size
        it_n += 1
        sub_n += sub_dat.shape[0]
        print(f'Grabbed iter {it_n} total sub n so far {sub_n}')
        ret_chunk = file_chunk.shape[0]
    fin_dat = pd.concat(fin_li_dat, axis=0)
    return fin_dat

In [3]:
end_str = [' STE', ' SUITE', ' BLDG', ' TOWER', ', #', ' UNIT',
           ' APT', ' BUILDING',',', '#']

def clean_add(address):
    add_new = address.upper()
    for su in end_str:
        sf = address.find(su)
        if sf > -1:
            add_new = add_new[0:sf]
    add_new = add_new.replace('.','')
    add_new = add_new.strip()
    return add_new

In [4]:
def split_geo(df, add, city, state, zipcode, chunk_size=500):
    df_new = df.copy()
    df_new.reset_index(inplace=True)
    splits = np.ceil(df_new.shape[0]/chunk_size)
    chunk_li = np.array_split(df_new['index'], splits)
    res_li = []
    pick_fi = []
    for i,c in enumerate(chunk_li):
        # Grab data, export to csv
        sub_data = df_new.loc[c, ['index',add,city,state,zipcode]]
        sub_data.to_csv('temp_geo.csv',header=False,index=False)
        # Geo the results and turn back into df
        print(f'Geocoding round {int(i)+1} of {int(splits)}, {datetime.now()}')
        result = cg.addressbatch('temp_geo.csv') #should try/except?
        # May want to dump the intermediate results
        #pi_str = f'pickres_{int(i)}.p'
        #pickle.dump( favorite_color, open( pi_str, "wb" ) )
        #pick_fi.append(pi_str.copy())
        names = list(result[0].keys())
        res_zl = []
        for r in result:
            res_zl.append(list(r.values()))
        res_df = pd.DataFrame(res_zl, columns=names)
        res_li.append(res_df.copy())
        # time.sleep(10) #sleep 10 seconds to not get cutoff from request
    final_df = pd.concat(res_li)
    final_df.rename(columns={'id':'row'}, inplace=True)
    final_df.reset_index(inplace=True, drop=True)
    # Clean up csv file
    os.remove('temp_geo.csv')
    return final_df

### California

In [5]:
def sub_rows(data):
    ec = data['Entity Type Code'] == "2"
    st = data['Provider Business Practice Location Address State Name'] == 'CA'
    ta = data[taxon_codes].isin(community_pharm).any(axis=1)
    #ac = data['NPI Deactivation Reason Code'].isna()
    all_together = ec & st & ta  #& ac 
    sub = data[all_together]
    return sub

print( datetime.now() )
pharm_tx = csv_chunks(npi_csv, chunk_size=1000000, keep_cols=keep_col, row_sub=sub_rows)
print( datetime.now() )
ph_tx_ca = pharm_tx.drop(columns=taxon_codes).reset_index(drop=True)

2023-04-03 06:18:29.926131
Grabbed iter 1 total sub n so far 67
Grabbed iter 2 total sub n so far 188
Grabbed iter 3 total sub n so far 340
Grabbed iter 4 total sub n so far 442
Grabbed iter 5 total sub n so far 523
Grabbed iter 6 total sub n so far 657
Grabbed iter 7 total sub n so far 754
Grabbed iter 8 total sub n so far 838
2023-04-03 06:20:34.365264


### Arizona

In [6]:
def sub_rows(data):
    ec = data['Entity Type Code'] == "2"
    st = data['Provider Business Practice Location Address State Name'] == 'AZ'
    ta = data[taxon_codes].isin(community_pharm).any(axis=1)
    # ac = data['NPI Deactivation Reason Code'].isna()
    all_together = ec & st & ta #& ac 
    sub = data[all_together]
    return sub

print( datetime.now() )
pharm_tx = csv_chunks(npi_csv, chunk_size=1000000, keep_cols=keep_col, row_sub=sub_rows)
print( datetime.now() )
ph_tx_az = pharm_tx.drop(columns=taxon_codes).reset_index(drop=True)

2023-04-03 06:20:34.371902
Grabbed iter 1 total sub n so far 7
Grabbed iter 2 total sub n so far 34
Grabbed iter 3 total sub n so far 51
Grabbed iter 4 total sub n so far 76
Grabbed iter 5 total sub n so far 100
Grabbed iter 6 total sub n so far 140
Grabbed iter 7 total sub n so far 176
Grabbed iter 8 total sub n so far 200
2023-04-03 06:22:36.383845


### Nevada

In [7]:
def sub_rows(data):
    ec = data['Entity Type Code'] == "2"
    st = data['Provider Business Practice Location Address State Name'] == 'NV'
    ta = data[taxon_codes].isin(community_pharm).any(axis=1)
    # ac = data['NPI Deactivation Reason Code'].isna()
    all_together = ec & st & ta #& ac 
    sub = data[all_together]
    return sub

print( datetime.now() )
pharm_tx = csv_chunks(npi_csv, chunk_size=1000000, keep_cols=keep_col, row_sub=sub_rows)
print( datetime.now() )

ph_tx_nv = pharm_tx.drop(columns=taxon_codes).reset_index(drop=True)

2023-04-03 06:22:36.388610
Grabbed iter 1 total sub n so far 4
Grabbed iter 2 total sub n so far 18
Grabbed iter 3 total sub n so far 28
Grabbed iter 4 total sub n so far 36
Grabbed iter 5 total sub n so far 46
Grabbed iter 6 total sub n so far 62
Grabbed iter 7 total sub n so far 78
Grabbed iter 8 total sub n so far 102
2023-04-03 06:26:50.017948


### Texas

In [8]:
def sub_rows(data):
    ec = data['Entity Type Code'] == "2"
    st = data['Provider Business Practice Location Address State Name'] == 'TX'
    ta = data[taxon_codes].isin(community_pharm).any(axis=1)
    # ac = data['NPI Deactivation Reason Code'].isna()
    all_together = ec & st & ta #& ac 
    sub = data[all_together]
    return sub

print( datetime.now() )
pharm_tx = csv_chunks(npi_csv, chunk_size=1000000, keep_cols=keep_col, row_sub=sub_rows)
print( datetime.now() )

ph_tx_tx = pharm_tx.drop(columns=taxon_codes).reset_index(drop=True)

2023-04-03 06:26:50.024822
Grabbed iter 1 total sub n so far 55
Grabbed iter 2 total sub n so far 188
Grabbed iter 3 total sub n so far 282
Grabbed iter 4 total sub n so far 360
Grabbed iter 5 total sub n so far 441
Grabbed iter 6 total sub n so far 570
Grabbed iter 7 total sub n so far 665
Grabbed iter 8 total sub n so far 746
2023-04-03 06:29:15.509278


### Washington

In [9]:
def sub_rows(data):
    ec = data['Entity Type Code'] == "2"
    st = data['Provider Business Practice Location Address State Name'] == 'WA'
    ta = data[taxon_codes].isin(community_pharm).any(axis=1)
    # ac = data['NPI Deactivation Reason Code'].isna()
    all_together = ec & st & ta #& ac 
    sub = data[all_together]
    return sub

print( datetime.now() )
pharm_tx = csv_chunks(npi_csv, chunk_size=1000000, keep_cols=keep_col, row_sub=sub_rows)
print( datetime.now() )

ph_tx_wa = pharm_tx.drop(columns=taxon_codes).reset_index(drop=True)

2023-04-03 06:29:15.514587
Grabbed iter 1 total sub n so far 15
Grabbed iter 2 total sub n so far 54
Grabbed iter 3 total sub n so far 92
Grabbed iter 4 total sub n so far 107
Grabbed iter 5 total sub n so far 126
Grabbed iter 6 total sub n so far 153
Grabbed iter 7 total sub n so far 191
Grabbed iter 8 total sub n so far 205
2023-04-03 06:31:14.689026


### Oregon

In [10]:
def sub_rows(data):
    ec = data['Entity Type Code'] == "2"
    st = data['Provider Business Practice Location Address State Name'] == 'OR'
    ta = data[taxon_codes].isin(community_pharm).any(axis=1)
    # ac = data['NPI Deactivation Reason Code'].isna()
    all_together = ec & st & ta #& ac 
    sub = data[all_together]
    return sub

print( datetime.now() )
pharm_tx = csv_chunks(npi_csv, chunk_size=1000000, keep_cols=keep_col, row_sub=sub_rows)
print( datetime.now() )

ph_tx_or = pharm_tx.drop(columns=taxon_codes).reset_index(drop=True)

2023-04-03 06:31:14.694511
Grabbed iter 1 total sub n so far 3
Grabbed iter 2 total sub n so far 22
Grabbed iter 3 total sub n so far 47
Grabbed iter 4 total sub n so far 62
Grabbed iter 5 total sub n so far 78
Grabbed iter 6 total sub n so far 90
Grabbed iter 7 total sub n so far 113
Grabbed iter 8 total sub n so far 124
2023-04-03 06:33:16.294914


### Illinios

In [11]:
def sub_rows(data):
    ec = data['Entity Type Code'] == "2"
    st = data['Provider Business Practice Location Address State Name'] == 'IL'
    ta = data[taxon_codes].isin(community_pharm).any(axis=1)
    # ac = data['NPI Deactivation Reason Code'].isna()
    all_together = ec & st & ta #& ac 
    sub = data[all_together]
    return sub

print( datetime.now() )
pharm_tx = csv_chunks(npi_csv, chunk_size=1000000, keep_cols=keep_col, row_sub=sub_rows)
print( datetime.now() )

ph_tx_il = pharm_tx.drop(columns=taxon_codes).reset_index(drop=True)

2023-04-03 06:33:16.298729
Grabbed iter 1 total sub n so far 29
Grabbed iter 2 total sub n so far 82
Grabbed iter 3 total sub n so far 114
Grabbed iter 4 total sub n so far 137
Grabbed iter 5 total sub n so far 190
Grabbed iter 6 total sub n so far 263
Grabbed iter 7 total sub n so far 329
Grabbed iter 8 total sub n so far 352
2023-04-03 06:35:18.643346


In [12]:
ph_tx = pd.concat([ph_tx_ca, ph_tx_az, ph_tx_nv, ph_tx_tx, ph_tx_wa, ph_tx_or, ph_tx_il])

In [13]:
ph_tx.to_csv('statesDf.csv',index=False)

In [14]:
len(ph_tx)

2567

In [15]:
ph_tx['Provider Business Practice Location Address Postal Code'] = ph_tx['Provider Business Practice Location Address Postal Code'].str[0:5]
ph_tx['Zip5'] = ph_tx['Provider Business Practice Location Address Postal Code'].str[0:5]
ph_tx['Address'] = ph_tx['Provider First Line Business Practice Location Address'].apply(clean_add)

ph_tx.rename(columns={'Provider Business Practice Location Address City Name':'City',
                      'Provider Business Practice Location Address State Name':'State2'},
             inplace=True)
ph_tx = ph_tx[-ph_tx['Provider Organization Name (Legal Business Name)'].str.contains("NORDSTROM")].reset_index(drop = True)
ph_tx.head(2)

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider First Line Business Practice Location Address,City,State2,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Telephone Number,Zip5,Address
0,1548468614,2,DIMENSION PROSTHETICS & ORTHOTICS,33374 DOWE AVE,UNION CITY,CA,94587,5103243400,94587,33374 DOWE AVE
1,1952507303,2,"SOUND BALANCE AUDIOLOGY, INC",2420 VISTA WAY,OCEANSIDE,CA,92054,7607217417,92054,2420 VISTA WAY


In [16]:
len(ph_tx)

2510

In [17]:
geo_pharm = split_geo(ph_tx, add='Address', city='City', state='State2', zipcode='Zip5', chunk_size=500)
print(geo_pharm['match'].value_counts())

Geocoding round 1 of 6, 2023-04-03 06:36:22.934412
Geocoding round 2 of 6, 2023-04-03 06:36:37.639451
Geocoding round 3 of 6, 2023-04-03 06:36:48.960664
Geocoding round 4 of 6, 2023-04-03 06:37:21.351670
Geocoding round 5 of 6, 2023-04-03 06:38:05.297011
Geocoding round 6 of 6, 2023-04-03 06:38:32.526470
True     2237
False     273
Name: match, dtype: int64


In [18]:
geo_pharm['rowN'] = geo_pharm['row'].astype(int)
gp2 = geo_pharm.sort_values(by='rowN').reset_index(drop=True)

kg = ['address','match','lat','lon']
kd = ['NPI',
      'Provider Organization Name (Legal Business Name)',
      'Provider Business Practice Location Address Telephone Number',
      'City','State2','Zip5']
final_pharm = pd.concat([ph_tx[kd], gp2[kg]], axis=1)

final_pharm.rename(columns={'Provider Organization Name (Legal Business Name)':'Name',
                      'Provider Business Practice Location Address Telephone Number':'Telephone'}, inplace=True)
final_pharm.head(2)

,NPI,Name,Telephone,City,State2,Zip5,address,match,lat,lon
0,1548468614,DIMENSION PROSTHETICS & ORTHOTICS,5103243400,UNION CITY,CA,94587,"33374 DOWE AVE, UNION CITY, CA, 94587",True,37.601059,-122.043522
1,1952507303,"SOUND BALANCE AUDIOLOGY, INC",7607217417,OCEANSIDE,CA,92054,"2420 VISTA WAY, OCEANSIDE, CA, 92054",True,33.183241,-117.334400


In [19]:
final_pharm.to_csv('clinics_States.csv',index=False)

In [20]:
hosp_data = pd.read_csv('clinics_States.csv')
hosp_data = hosp_data[hosp_data['match']].copy()
hosp_data.reset_index(inplace=True, drop=True)
hosp_data.head(2)

,NPI,Name,Telephone,City,State2,Zip5,address,match,lat,lon
0,1548468614,DIMENSION PROSTHETICS & ORTHOTICS,5103243400,UNION CITY,CA,94587,"33374 DOWE AVE, UNION CITY, CA, 94587",True,37.601059,-122.043522
1,1952507303,"SOUND BALANCE AUDIOLOGY, INC",7607217417,OCEANSIDE,CA,92054,"2420 VISTA WAY, OCEANSIDE, CA, 92054",True,33.183241,-117.334400


In [21]:
hosp_geo = gpd.GeoDataFrame(hosp_data, geometry=gpd.points_from_xy(hosp_data.lon, hosp_data.lat), crs="EPSG:4326")

In [22]:
cali_counties = gpd.read_file(r'tl_2019_us_county/tl_2019_us_county.shp')
cali_outline = cali_counties.dissolve('STATEFP')
cali_proj = cali_outline.to_crs('EPSG:5070')
print(cali_outline.crs)

EPSG:4269


In [23]:
def dissolve_buff(point_df,d,resolution):
    bu = point_df.buffer(d,resolution)
    geodf = gpd.GeoDataFrame(geometry=bu)
    geodf['Const'] = 0
    single = geodf.dissolve('Const')
    return single[['geometry']]

In [24]:
def dist_cont(point_df,dist_list,outside,buff_res):
    if point_df.crs != outside.crs:
        print('Point df and Outside df are not the same CRS')
        return None
    # Making outside area out dissolved object
    out_cop = outside[['geometry']].copy()
    out_cop['Constant'] = 1
    out_cop = out_cop.dissolve('Constant')
    # Make sure points are inside area
    inside = point_df.within(out_cop['geometry'][1])
    point_cop = point_df[inside].copy()
    point_cop = point_df.copy()
    point_cop['Constant'] = 1 #Constant for dissolve
    point_cop = point_cop[['Constant','geometry']].copy()
    res_buffers = []
    for i,d in enumerate(dist_list):
        print(f'Doing buffer {d}')
        if i == 0:
            res = dissolve_buff(point_cop, d, buff_res)
            res_buffers.append(res.copy())
        else:
            res_new = dissolve_buff(point_cop, d, buff_res)
            res_buffonly = gpd.overlay(res_new, res, how='difference')
            res = res_new.copy()
            res_buffers.append( res_buffonly.copy() )
    # Now take the difference with the larger area
    print('Working on leftover difference now')
    leftover = gpd.overlay(out_cop, res, how='difference')
    res_buffers.append(leftover)
    for i,d in enumerate(dist_list):
        res_buffers[i]['Distance'] = str(d)
    res_buffers[-1]['Distance'] = 'Outside'
    # New geopandas DF
    comb_df = pd.concat(res_buffers)
    comb_df.reset_index(inplace=True, drop=True)
    return comb_df

In [25]:
hos_proj = hosp_geo.to_crs('EPSG:5070') #'epsg:4269'

dist_met = [2000, 4000, 8000, 16000] #, 32000
buff_city = dist_cont(hos_proj, dist_met, cali_proj, buff_res=100)

Doing buffer 2000
Doing buffer 4000
Doing buffer 8000
Doing buffer 16000
Working on leftover difference now


In [26]:
#Now making folium plot
buff_map = buff_city.to_crs('EPSG:4326')
kv = list(hosp_geo)[1:10]

In [27]:
#"fill": "#00aa22",
#"fill-opacity": 0.5

cols = ['#f1eef6',
'#d7b5d8',
'#df65b0',
'#dd1c77',
'#980043']

buff_map['fill'] = cols
buff_map['fill-opacity'] = 0.35

#os.chdir(r'D:\Dropbox\Dropbox\PublicCode_Git\Blog_Code')

In [28]:
buff_map.to_file('Buffers_States.geojson', driver='GeoJSON')
hosp_geo.to_file('Hosp_States.geojson', driver='GeoJSON')